<a href="https://colab.research.google.com/github/GerardoFesta/3DFER_SE4AI/blob/main/LandmarkMLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os 
from google.colab import drive
drive.mount("/content/drive")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip "training_set.csv.zip"

Archive:  training_set.csv.zip
  inflating: training_set.csv        
  inflating: __MACOSX/._training_set.csv  


In [ ]:

import sys
import pandas as pd
import torch
import torch.nn.functional as F
from torch import nn 
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import torch.optim as optim
import numpy as np

In [ ]:
train_df = pd.read_csv("training_set.csv")
test_df = pd.read_csv("test_set.csv")

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#Questo commento serve per provare il funzionamento dei commit con colab

batch_size = 1024 

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, X_array, Y_array, transform=None):
        self.X = X_array
        self.Y = Y_array
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        image = self.X[index]
        label = self.Y[index] 

        # Esegui le trasformazioni se definite
        if self.transform is not None:
            image = self.transform(image)

        return image, label

In [ ]:

test_df['landmarks'] = test_df['landmarks'].apply(lambda lab: eval(lab))

train_df['landmarks'] = train_df['landmarks'].apply(lambda lab: eval(lab))
print(train_df)

                                      nome  \
0        train/angry/Training_12391352.jpg   
1        train/angry/Training_52407046.jpg   
2        train/angry/Training_20666200.jpg   
3        train/angry/Training_11533347.jpg   
4        train/angry/Training_60003551.jpg   
...                                    ...   
26700  train/disgust/Training_63164084.jpg   
26701  train/disgust/Training_25610374.jpg   
26702  train/disgust/Training_67023235.jpg   
26703  train/disgust/Training_61032772.jpg   
26704  train/disgust/Training_96306068.jpg   

                                               landmarks    label  
0      [[0.5751925706863403, 0.5731657147407532, -0.1...    angry  
1      [[0.43962034583091736, 0.7553703784942627, -0....    angry  
2      [[0.6029075384140015, 0.5643556118011475, -0.0...    angry  
3      [[0.5550940036773682, 0.7429446578025818, -0.0...    angry  
4      [[0.5235995054244995, 0.8211154937744141, -0.0...    angry  
...                                    

In [ ]:
print(type(train_df.at[0,"landmarks"]))

label_dict = {"angry":0, "sad": 1, "neutral": 2, "surprise": 3, "disgust": 4, "fear": 5, "happy": 6}

test_df['label'] = test_df['label'].apply(lambda lab: label_dict[lab])

train_df['label'] = train_df['label'].apply(lambda lab: label_dict[lab])

array_train = train_df['landmarks'].to_numpy()
X_train = np.stack([np.array(lst) for lst in array_train])
y_train = train_df['label'].to_numpy()
array_test = test_df['landmarks'].to_numpy()
X_test = np.stack([np.array(lst) for lst in array_test])
y_test = test_df['label'].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(np.r_[X_train, X_test], np.r_[y_train, y_test], test_size=0.33, random_state=42, stratify=np.r_[y_train, y_test])

train_dataset = CustomDataset(X_train, y_train, transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


test_dataset = CustomDataset(X_test, y_test, transform=transforms.ToTensor())

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print( X_train.shape, y_train.shape, X_test.shape, y_test.shape)

<class 'list'>
(22366, 478, 3) (22366,) (11017, 478, 3) (11017,)


In [ ]:
def accuracy(preds, labels):
  probabilities = torch.nn.functional.softmax(preds, dim=1)
  _, predicted = torch.max(probabilities, dim=1)
  n_correct = (predicted==labels).sum().float()

  acc =n_correct / labels.shape[0]
  acc= torch.round(acc*100)
  return acc, n_correct;

In [ ]:
class MultiClassificator(nn.Module):
  def __init__(self, in_size: int, hidden_size: int, num_classes: int):
    super(MultiClassificator, self).__init__()

    self.fc1 = nn.Linear(in_size, hidden_size, dtype=torch.float64)
    self.fc2 = nn.Linear(hidden_size, num_classes, dtype=torch.float64)

  def forward(self, x: torch.Tensor):
    b = x.shape[0]
    x = x.view(b,-1)
    out_1 = self.fc1(x)

    out_2 = F.tanh(out_1)
    out_2_2 = F.tanh(out_2)

    out_3 = self.fc2(out_2)

    return out_3

In [ ]:
model = MultiClassificator(478*3, 1024,  7)
print(model)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

MultiClassificator(
  (fc1): Linear(in_features=1434, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=7, bias=True)
)


In [ ]:
model.train()
num_epochs = 20
for epoch in range(num_epochs):  

    running_loss = 0.0
    running_acc = 0.0
    tot_seen = 0
    for X_batch, y_batch in train_loader:

        
        #manda i batch al device 
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        #azzera gradiente
        optimizer.zero_grad()
        #predict
        y_pred = model(X_batch)
        #calcola loss e accuracy
        #y_pred_test = torch.sigmoid(y_pred)
       
        #y_pred_test = [1 if x>=.5 else 0 for x in y_pred_test]
      

        #print(y_batch.shape)
        loss = criterion(y_pred, y_batch)
        _,acc = accuracy(y_pred, y_batch)
        #backpropagation della loss
        loss.backward()
        #ottimizzazione
        optimizer.step()
        
        #somma della loss e dell'accuracy per il batch
        running_loss += loss.item()
        running_acc += acc
        tot_seen += len(y_batch)                    
        
    
    print(f'Epoch {epoch}: | Loss: {running_loss/len(train_loader):.5f} | Acc: {running_acc/tot_seen:.3f}')


Epoch 0: | Loss: 2.11942 | Acc: 0.204
Epoch 1: | Loss: 1.79751 | Acc: 0.257
Epoch 2: | Loss: 1.76059 | Acc: 0.291
Epoch 3: | Loss: 1.72755 | Acc: 0.318
Epoch 4: | Loss: 1.67774 | Acc: 0.355
Epoch 5: | Loss: 1.61441 | Acc: 0.389
Epoch 6: | Loss: 1.54485 | Acc: 0.420
Epoch 7: | Loss: 1.50872 | Acc: 0.424
Epoch 8: | Loss: 1.50180 | Acc: 0.423
Epoch 9: | Loss: 1.47528 | Acc: 0.435
Epoch 10: | Loss: 1.43018 | Acc: 0.453
Epoch 11: | Loss: 1.41438 | Acc: 0.461
Epoch 12: | Loss: 1.40955 | Acc: 0.462
Epoch 13: | Loss: 1.41604 | Acc: 0.457
Epoch 14: | Loss: 1.40596 | Acc: 0.463
Epoch 15: | Loss: 1.39079 | Acc: 0.470
Epoch 16: | Loss: 1.39520 | Acc: 0.466
Epoch 17: | Loss: 1.38427 | Acc: 0.474
Epoch 18: | Loss: 1.37891 | Acc: 0.473
Epoch 19: | Loss: 1.41595 | Acc: 0.456


In [ ]:
y_pred_list = []
tot_corrette=0
n_seen=0
running_loss=0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        y_pred = model(X_batch)
        _, n_corrette =accuracy(y_pred,y_batch)
        tot_corrette+=n_corrette
        n_seen+=len(y_pred)
        loss = criterion(y_pred, y_batch)
        running_loss += loss.item()
    print(f'Acc: {tot_corrette/n_seen:.3f}, Loss:{running_loss/len(test_loader)}')

Acc: 0.473, Loss:1.371220305255462
